Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Unzip the dataset

In [2]:
import zipfile

In [3]:
dataset_path = '/content/drive/MyDrive/skin cancer dataset.zip'
extracted_path = '/content/dataset'

In [4]:
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

In [ ]:
#!ls '/content/dataset'

In [5]:
import os
import numpy as np
import tensorflow as tf
import PIL.Image

In [6]:
# Define the CNN architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])


In [7]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [8]:
# Resize the training images to 224x224 pixels
def resize_image(image):
    return image.resize((224, 224))

# Split the training images into smaller batches
batch_size = 320

In [9]:
# Load the training images and labels
training_images = []
training_labels = []

for root, directories, files in os.walk('/content/dataset/melanoma_cancer_dataset/train/benign'):
    for file in files:
        image = PIL.Image.open(os.path.join(root, file))
        image = resize_image(image)

        training_images.append(image)
        training_labels.append(0)

for root, directories, files in os.walk('/content/dataset/melanoma_cancer_dataset/train/malignant'):
    for file in files:
        image = PIL.Image.open(os.path.join(root, file))
        image = resize_image(image)

        training_images.append(image)
        training_labels.append(1)


In [10]:
# Convert the training labels to a NumPy array
training_labels = np.array(training_labels)

In [11]:
# Convert the training images to a NumPy array
training_images = np.array([np.array(image) for image in training_images])

In [ ]:
#  Train the model
model.fit(training_images, training_labels, batch_size=32, epochs=1)


301/301 [==============================] - 845s 3s/step - loss: 45.2720 - accuracy: 0.6815


In [14]:
# Load the test images and labels
test_images = []
test_labels = []

for root, directories, files in os.walk('/content/dataset/melanoma_cancer_dataset/test/benign'):
    for file in files:
        image = PIL.Image.open(os.path.join(root, file))
        image = resize_image(image)

        test_images.append(image)
        test_labels.append(0)

for root, directories, files in os.walk('/content/dataset/melanoma_cancer_dataset/test/malignant'):
    for file in files:
        image = PIL.Image.open(os.path.join(root, file))
        image = resize_image(image)

        test_images.append(image)
        test_labels.append(1)

In [15]:
# Convert the test labels to a NumPy array
test_labels = np.array(test_labels)

In [16]:
# Convert the test images to a NumPy array
test_images = np.array([np.array(image) for image in test_images])

In [17]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)


32/32 [==============================] - 22s 664ms/step - loss: 0.5224 - accuracy: 0.7440


In [18]:

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

Test Loss: 0.5224368572235107
Test Accuracy: 0.7440000176429749


In [19]:
# Save the trained model
model.save('melanoma_cancer_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
